<b>Train Loss function: Mean Squared Error

In [2]:
#Import python libraries
import numpy as np
from sklearn.cross_validation import train_test_split
import theano
import theano.tensor as T
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
np.random.seed(12)
rng = np.random

In [3]:
#Generate X, Y data (random floats or intergers, comment code as needed)
N = 500 #Number of observations
X = rng.randn(N)
Y = rng.randn(N)
#X = rng.randint(size=N, low=0, high=10)
#Y = rng.randint(size=N, low=0, high=10)

In [4]:
#Generate Z data based on the given analytic function 
Z = 2*pow(X,2) - 3*pow(Y,2) + 1
Z

array([ -8.46152815e-02,  -4.47512231e+00,  -4.80323747e+00,
         6.35520690e+00,  -1.25878318e+00,   3.79381767e+00,
         9.36679493e-01,  -2.49754833e+00,   1.78470768e+00,
         1.42776087e+01,   1.56609368e+00,  -1.39279471e+00,
         3.31579779e+00,   3.87163537e+00,   4.08208255e-01,
         1.90423558e-02,   3.00864677e+00,   4.39203818e-01,
         2.29866728e+00,   3.63107994e+00,  -1.30140127e+01,
        -1.44250524e+00,   1.72308061e+00,  -9.52256790e-01,
         3.65898315e+00,   8.56203335e+00,   4.37595468e+00,
         6.78029653e+00,   5.71523174e+00,   1.83508637e+00,
         9.91725723e-02,  -1.78818887e+00,  -2.28931599e+00,
         2.61022632e-01,  -4.13188124e+00,  -2.06773768e+00,
         2.63162030e+00,   1.69434899e+00,   6.83315324e-01,
         1.87451368e+00,  -5.98813923e+00,   4.71261370e-01,
         7.68735255e-01,   4.75614129e-01,  -3.59626450e+00,
        -6.06675137e-01,  -9.34435885e+00,   1.06944714e+01,
         1.09532301e+01,

In [5]:
#Split data into training and test sets
X_train, X_test, Y_train, Y_test, Z_train, Z_test = train_test_split(
     X, Y, Z, test_size=0.33, random_state=42)

In [6]:
#Initialize coefficients and declare theano symbolic variables for Mean Squared Error (MSE)
m1_value = rng.randn() #coefficient of x
m2_value = rng.randn() #coefficient of y
b_value = rng.randn()  #bias

m1 = theano.shared(m1_value,name ='m1')
m2 = theano.shared(m2_value,name ='m2')
b = theano.shared(b_value,name ='b')

x = T.vector('x')
y = T.vector('y')
z = T.vector('z')

num_samples = X.shape[0]

In [7]:
#Define the loss function for MSE
prediction = T.dot(x,m1) + T.dot(y,m2) + b
loss = T.sum(T.pow(prediction-z,2))/(2*num_samples) 
#loss1 = T.sum(abs(prediction-z))/(2*num_samples) #MAE Loss Function

In [8]:
#Compute the gradient of our loss function(s)
gradm1 = T.grad(loss,m1)
gradm2 = T.grad(loss,m2)
gradb = T.grad(loss,b)

In [9]:
#Optimization, Make our function with a learning rate of 0.01 and return the cost
learning_rate = 0.01
train = theano.function([x,y,z], 
                        [loss, m1-learning_rate*gradm1, m2-learning_rate*gradm2, b-learning_rate*gradb],
                         updates = [(m1,m1-learning_rate*gradm1), (m2,m2-learning_rate*gradm2),(b,b-learning_rate*gradb)])
test = theano.function([x,y],prediction)

In [10]:
#Train the model for 500 steps and print parameters with least amount of cost
training_steps = 500
costF = []
for i in range(training_steps):
    costM = train(X_train,Y_train,Z_train)
    costF.append(costM)
    #print(costM)

print("Slope 1:")
print(m1.get_value())
print("Slope 2:")
print(m2.get_value())
print("Intercept :")
print(b.get_value())

Slope 1:
-0.992406941129
Slope 2:
0.776751413811
Intercept :
-0.489557705562


In [11]:
#Display cost, coefficients (m1 & m2) and bias value from 500 iterations
costF

[[array(14.273623937813424),
  array(1.8691321751585084),
  array(1.72744770711354),
  array(-0.03139897658208358)],
 [array(14.229497346402404),
  array(1.8492509703733815),
  array(1.7210970168526496),
  array(-0.03251224967800305)],
 [array(14.18596047957529),
  array(1.829504685454889),
  array(1.7147865585702657),
  array(-0.03363388721435813)],
 [array(14.143005352888231),
  array(1.8098923903471855),
  array(1.7085160829201507),
  array(-0.034763725525029154)],
 [array(14.100624091407026),
  array(1.790413161506072),
  array(1.7022853420478137),
  array(-0.03590160278472212)],
 [array(14.058808928186151),
  array(1.7710660818527164),
  array(1.6960940895821044),
  array(-0.03704735899140283)],
 [array(14.0175522027691),
  array(1.7518502407277037),
  array(1.6899420806268501),
  array(-0.038200835948886125)],
 [array(13.97684635970988),
  array(1.73276473384542),
  array(1.683829071752534),
  array(-0.03936187724957886)],
 [array(13.936683947115238),
  array(1.7138086632487641),

In [12]:
#Create individual variables for error surface plot
costval = [item[0] for item in costF]
m1val = [item[1] for item in costF]
m2val = [item[2] for item in costF]
bval = [item[3] for item in costF]
#m1val

In [13]:
#Plot squared error surface area
fig = plt.figure()
#ax = Axes3D(fig)
X1, Y1 = np.meshgrid(m1val, m2val)
ax = fig.gca(projection='3d')
ax.plot_surface(X1,Y1,costval,rstride=1, cstride=1, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
plt.show()

<img src="sq_err_surface_plot.png">

In [13]:
#Function to calculate accuracy peformance measure
def accuracy(predicted, actual):
    total = 0.0
    correct = 0.0
    for p, a in zip(predicted, actual):
        total += 1
        if p == a:
            correct += 1
    return correct / total

In [14]:
#Predict values based on test set
pred = test(X_test, Y_test)

In [15]:
#Display accuracy rate
accuracy(pred.astype(int), Z_test.astype(int))

0.2727272727272727